In [1]:

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import load_model

from tqdm import tqdm

In [2]:
from utils import utils

In [3]:
RANDOM_STATE = 35

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Preprocessing

In [4]:
data_ts = pd.read_csv("dataset/processed_features.csv")


columns=['account_id', 'is_fraud', 'device_id', 'balance', 'processed_at',
       'age_range', 'number_of_selfies_sent', 'time_client', 'cash_out_type_1',
       'cash_out_type_2', 'cash_out_type_3', 'cash_out_type_6']

data_ts = data_ts[columns]

# the columns that pass the float16 limite, we gonna take them and turn and something
# that we can work using min max scaler

max_account = data_ts["account_id"].max()
max_device_id = data_ts["device_id"].max()
max_processed_at = data_ts["processed_at"].max()
max_time_client = data_ts["time_client"].max()

data_ts["account_id"] = data_ts["account_id"] / max_account
data_ts["device_id"] = data_ts["device_id"] / max_device_id
data_ts["processed_at"] = data_ts["processed_at"] / max_processed_at
data_ts["time_client"] = data_ts["time_client"] / max_time_client

# seconds to days

data_ts.time_client = data_ts.time_client / (60*60*24)

data_ts.head()

,account_id,is_fraud,device_id,balance,processed_at,age_range,number_of_selfies_sent,time_client,cash_out_type_1,cash_out_type_2,cash_out_type_3,cash_out_type_6
0,0.000000,0,0.0,0.000167,0.999813,0.0,5.0,1.785904e-07,1,0,0,0
1,0.000016,0,0.0,0.000533,0.998152,1.0,5.0,1.970170e-06,0,0,1,0
2,0.000016,0,0.0,0.000347,0.999702,1.0,5.0,2.456546e-06,0,0,1,0
3,0.000016,0,0.0,0.000572,0.998524,1.0,5.0,2.087007e-06,0,0,1,0
4,0.000016,0,0.0,0.000346,0.998629,1.0,5.0,2.119767e-06,0,0,1,0


In [5]:
#droping the target and the features weakly correlated to the target
data_to_encode = data_ts.copy().drop(["account_id", "processed_at", "number_of_selfies_sent", "cash_out_type_6"], axis=1)

Y = np.array(data_to_encode.pop("is_fraud"), dtype=np.float32)
X = np.array(data_to_encode, dtype=np.float32)

# Model

In [6]:
encoder = load_model("../api_fraud_detection/model/saved_models/bottleneck")

2021-12-13 18:56:37.997610: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
train, test = train_test_split(data_ts, test_size = 0.01,  stratify=data_ts.is_fraud)
train, val = train_test_split(train, test_size = 0.01, stratify=train.is_fraud)

test.to_csv("test_df.csv", index=False)

train.drop(["account_id", "processed_at","number_of_selfies_sent",
            "cash_out_type_6"], inplace=True, axis=1)
val.drop(["account_id", "processed_at","number_of_selfies_sent",
            "cash_out_type_6"], inplace=True, axis=1)
test.drop(["account_id", "processed_at","number_of_selfies_sent",
            "cash_out_type_6"], inplace=True, axis=1)

labels_test = np.array(test.pop("is_fraud")) 
labels_train = np.array(train.pop("is_fraud"))
labels_val = np.array(val.pop("is_fraud"))


features_train = encoder.predict(np.array(train))
features_val = encoder.predict(np.array(val))
features_test = encoder.predict(np.array(test))

clf = RandomForestClassifier(max_depth= 50, max_features= 'log2', min_samples_split= 2, 
                            n_estimators= 400, random_state=RANDOM_STATE, n_jobs= 3,
                            class_weight= "balanced_subsample", min_samples_leaf= 2, 
                            verbose=1, oob_score=True)

clf.fit(features_train, labels_train)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:  4.3min
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed: 11.9min finished


RandomForestClassifier(class_weight='balanced_subsample', max_depth=50,
                       max_features='log2', min_samples_leaf=2,
                       n_estimators=400, n_jobs=3, oob_score=True,
                       random_state=35, verbose=1)

In [ ]:
X_pack = features_train, features_val
Y_pack = labels_train, labels_val

history = utils.compare_models(X_pack, Y_pack, clf)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


In [ ]:
plt.figure()
utils.plot_pr_curve(history)
plt.show

# Testing

In [ ]:
X_pack_test = None, features_test
Y_pack_test = None, labels_test

history_test = utils.compare_models(X_pack_test, Y_pack_test, clf)

In [ ]:
utils.plot_cm(labels_test, history_test["clf_0"]["y_hat_test"], "Final Model")

# K-fold cross-val

In [ ]:
encoded_x = np.concatenate((features_train, features_val), axis=1)
Y = np.concatenate((labels_train, labels_val), axis=1)

indices_split, (X_test, Y_test) = utils.split_data(encoded_x, Y,test_size=0.01,kfold=10,
                                                                    random_state=RANDOM_STATE)

for key, value in indices_split.items():
    
    print(f"{key} size train: {len(value[0])}, val: {len(value[1])}")

In [ ]:
clf_2 = RandomForestClassifier(max_depth= 50, max_features= 'log2', min_samples_split= 2, 
                                    n_estimators= 400, random_state=RANDOM_STATE, n_jobs= 3,
                                    class_weight= "balanced_subsample", min_samples_leaf= 2, 
                                    verbose=1, oob_score=True)

clfs = utils.train_model(clf_2, encoded_x, Y, indices_split)

In [ ]:
history_cross = compare_models(encoded_x, Y, clfs, indices_split)

In [ ]:
utils.plot_pr_curve(history_cross)

In [ ]:
from joblib import dump

dump(clf, 'model_rfc.joblib') 